In [ ]:
# 装入指定的模块
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np 
import torch
import pandas as pd
from scipy import io 
from torchvision import transforms
# Set up data loaders
from datasets import TripletMNIST, load_salinas_mat

from trainer import fit
cuda = torch.cuda.is_available()
ignored_labels = [0]
patch_size = 1
data_path = './data/Salinas/Salinas.mat'
gt_path = './data/Salinas/Salinas_gt.mat'
train_data, train_gt = load_salinas_mat(data_path, gt_path, remove_bands=True) # img:(H, W, d), gt:(H, W)
#train_data = io.loadmat(data_path)['salinas']
train_gt0 = io.loadmat(gt_path)['salinas_gt']
train_gt0 = train_gt0.astype(np.float32)
train_gt0 = train_gt0.flatten()
# train_gt0 = list(train_gt0)
# train_gt0 = train_gt0.tolist()
# Delete ndarray 0 elements
train_gt0 = pd.DataFrame(train_gt0).replace(0,np.NAN)
train_gt0.dropna(inplace = True)
train_gt0 = np.array(train_gt0).astype(np.float32).reshape(1,-1)[0]

train_gt = train_gt.astype(np.float32)
train_gt = train_gt.flatten()
n_classes = len(np.unique(train_gt))-1

train_data = np.float32(train_data)
max1 = np.amax(train_data)
min1 = np.amin(train_data)
train_data = (train_data-min1)/max1

d = train_data.shape[2]
train_data = train_data.reshape(-1, d)




triplet_train_dataset = TripletMNIST(train_data, train_gt, train_gt0) # Returns triplets of images

#triplet_test_dataset = TripletMNIST(test_dataset)
batch_size = 128
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
triplet_test_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=False, **kwargs)

# Set up the network and training parameters
from networks import FECNN, TripletNet
from losses import TripletLoss
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

margin = 1.
embedding_net = FECNN()
model = TripletNet(embedding_net)
if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 50, gamma=0.1, last_epoch=-1)
n_epochs = 50   #好的效果是50  Urabn=100
log_interval = 1


In [ ]:
fit(triplet_train_loader,triplet_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)